In [14]:
!pip install --upgrade azureml-sdk
!pip install --upgrade azureml-featurestore

In [12]:
from azureml.core import Workspace, Dataset

# Connect to your Azure ML workspace
ws = Workspace.from_config()

# Define connection details for Azure Synapse (using Azure SQL for demonstration)
datastore = ws.get_default_datastore()

# Register the dataset without copying it, referencing the Synapse SQL database
dataset = Dataset.Tabular.from_sql_query(
    datastore,
    query='SELECT * FROM your_synapse_table'
)

# Register the dataset in the Azure ML workspace for reuse
dataset = dataset.register(workspace=ws, name='SynapseDataset', create_new_version=True)


2024-09-14 21:41:37.572326 | ActivityCompleted: Activity=from_sql_query, HowEnded=Failure, Duration=0.14 [ms], Info = {'activity_id': '1afaca12-6b7e-4c4e-9769-53f125369523', 'activity_name': 'from_sql_query', 'activity_type': 'PublicApi', 'app_name': 'TabularDataset', 'source': 'azureml.dataset', 'version': '1.56.0', 'dataprepVersion': '5.1.6', 'sparkVersion': '', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': ''}, Exception=TypeError; TabularDatasetFactory.from_sql_query() got multiple values for argument 'query'


TypeError: TabularDatasetFactory.from_sql_query() got multiple values for argument 'query'

In [15]:
from azureml.featurestore import FeatureSet

# Retrieve the registered dataset from Azure ML
dataset = Dataset.get_by_name(workspace=ws, name='SynapseDataset')

# Convert the dataset to pandas dataframe (if needed for further processing)
df_synapse = dataset.to_pandas_dataframe()

# Define the schema for the feature set
feature_set_schema = {
    "feature1": DataType.FLOAT,
    "feature2": DataType.INT,
    "feature3": DataType.STRING,
}

# Ingest the data as a feature set without physically moving it
feature_set = FeatureSet(name="synapse_feature_set", version="1.0", schema=feature_set_schema)
fs_client.ingest(feature_set, df_synapse)

# Optionally, register the feature set in the feature store
feature_set.register()


ModuleNotFoundError: No module named 'azureml.featurestore'

In [ ]:
# Retrieve the features from the feature store
features = fs_client.get_feature_set("synapse_feature_set", version="1.0")

# Convert features to pandas DataFrame
df_features = features.to_pandas()

# Train a scikit-learn model using the retrieved data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = df_features.drop(columns=["label"])
y = df_features["label"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Train the model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
